# 2.1.3 Element-wise BDDC Preconditioner

The element-wise BDDC preconditioner in NGSolve is a good general purpose preconditioner that works well both in the shared memory parallel mode as well as in distributed memory mode. In this tutorial, we discuss this preconditioner, related built-in options, and customization from python.   

A simple description of the element-wise BDDC (**B**alancing **D**omain **D**ecomposition preconditioner with **C**onstraints) can be given in the context of Lagrange finite element space $V$: the BDDC preconditoner is constructed on an auxiliary space $\widetilde{V}$ obtained by connecting only element vertices (leaving edge and face shape functions discontinuous). Although larger, the auxiliary space allows local elimination of edge and face variables. Hence an analogue of the  original matrix $A$ on this space, named $\widetilde A$, is less expensive to invert.  This inverse is used to construct a preconditioner for $A$ as follows:

$$
C_{BDDC}^{-1} = R {\,\widetilde{A}\,}^{-1}\, R^t
$$

Here, $R$ is the averaging operator for the discontinous edge and face variables. 

To construct a general purpose BDDC preconditioner, NGSolve generalizes this idea to all its finite element spaces by a classification of degrees of freedom. NGSolve classifies dofs into (condensable) `LOCAL_DOF`s and a remainder that consists of these types: 

`WIREBASKET_DOF`  
`INTERFACE_DOF`

*The original finite element space $V$ is obtained by requiring conformity of both the above types of dofs, while the auxiliary space $\widetilde{V}$ is obtained by requiring conformity of `WIREBASKET_DOF`s only.*


In [1]:
import netgen.gui
# %gui tk
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner
from netgen.csg import unit_cube
from netgen.geom2d import unit_square
SetHeapSize(100*1000*1000)

In [2]:
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.5))
# mesh = Mesh(unit_square.GenerateMesh(maxh=0.5))

## Built-in options

Let us define a simple function to study how the spectrum of the preconditioned matrix changes with various options.

In [3]:
def TestPreconditioner (p, condense=False, **args):
    fes = H1(mesh, order=p, **args)
    u,v = fes.TnT()
    a = BilinearForm(fes, eliminate_internal=condense)
    a += SymbolicBFI(grad(u)*grad(v) + u*v)
    c = Preconditioner(a, "bddc")
    a.Assemble()   
    return EigenValues_Preconditioner(a.mat, c.mat)

In [4]:
lams = TestPreconditioner(5)
print (lams[0:3], "...\n", lams[-3:])

 1.00145
 1.05935
 1.23239
 ...
  21.2736
 22.7821
 22.9382



Here is the effect of static condensation on the BDDC preconditioner.

In [5]:
lams = TestPreconditioner(5, condense=True)
print (lams[0:3], "...\n", lams[-3:])

 1.00026
 1.03169
 1.09454
 ...
  4.10312
 4.13286
 4.24601



Next, let us study the effect of a few built-in flags for *finite element spaces* that are useful for tweaking the behavior of the BDDC preconditioner. The effect of these flags varies in two (2D) and three dimensions (3D), e.g., 

- `wb_fulledges=True`: This option keeps **all** edge-dofs conforming (i.e. they are marked `WIREBASKET_DOF`s). This option is only meaningful in 3D. If used in 2D, the preconditioner becomes a direct solver.

- `wb_withedges=True`: This option keeps only the **first** edge-dof conforming (i.e., the first edge-dof is marked `WIREBASKET_DOF` and the remaining edge-dofs are marked `INTERFACE_DOF`s). This is the default in 3D.

The complete situation is a bit more complex due to the fact these options  can take three values (True, False, Undefined), can be combined, and space dimension can be 2 or 3. So here is a table with the summary of the effect of these options:

| wb_fulledges      |  wb_withedges |  2D    |  3D    |
|-------------------|---------------|--------|--------|
|   True            |   any value   |  all   |   all  |
|   False/Undefined |   Undefined   |  none  |  first |
|   False/Undefined |    False      |  none  |  none  |
|   False/Undefined |    True       |  first |  first |

An entry $X \in$ {all, none, first} of the last two columns is to be read as follows: $X$ of the edge-dofs is(are) `WIREBASKET_DOF`(s).

Here is an example of the effect of one of these flag values.

In [6]:
lams = TestPreconditioner(5, condense=True, wb_withedges=False)
print (lams[0:3], "...\n", lams[-3:])

 1.00121
  1.0817
 1.23231
 ...
  25.7229
  25.723
 27.2163



Clearly, the conditioning became less favorable compared to the default. 

## Customize 

From within python, we can change the types of degrees of freedom of finite element spaces, thus affecting the behavior of the BDDC preconditioner. 

You can view the types of dofs in a finite element space as follows.

In [7]:
fes = H1(mesh, order=3)
print ("number vertices =", mesh.nv)
print ("dofs of first edge: ", fes.GetDofNrs(NodeId(EDGE,0)))
for i in range(fes.ndof):
    print ("ct[",i,"] = ", fes.CouplingType(i) )

number vertices = 21
dofs of first edge:  (21, 22)
ct[ 0 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 1 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 2 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 3 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 4 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 5 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 6 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 7 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 8 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 9 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 10 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 11 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 12 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 13 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 14 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 15 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 16 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 17 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 18 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 19 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 20 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 21 ] =  COUPLING_TYPE.WIREBASKET_DOF
ct[ 22 ] =  COUPLING_TYPE.INTERFACE_DOF
ct[ 23 ] =  COUPL

In [8]:
lams = TestPreconditioner (8, condense=True)
max(lams)/min(lams)

9.876864325521144

We may modify the dof types as follows.

In [9]:
fes = H1(mesh, order=8)
u,v = fes.TnT()

for ed in mesh.edges:
    dofs = fes.GetDofNrs(ed)
    for d in dofs:
        fes.SetCouplingType(d, COUPLING_TYPE.INTERFACE_DOF)

    # Set the first two edge dofs to be conforming
    fes.SetCouplingType(dofs[0], COUPLING_TYPE.WIREBASKET_DOF)
    fes.SetCouplingType(dofs[1], COUPLING_TYPE.WIREBASKET_DOF)

a = BilinearForm(fes, eliminate_internal=True)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
c = Preconditioner(a, "bddc")
a.Assemble()

lams=EigenValues_Preconditioner(a.mat, c.mat)
max(lams)/min(lams)

6.717426419118481